In [4]:
import numpy as np
import pandas as pd
from pandasql import sqldf

In [9]:
df_orders = pd.read_csv('instacart-market-basket-analysis/order_products__prior.csv')

In [10]:
df_orders.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [12]:
df_orders.shape

(32434489, 4)

In [14]:
df_product = pd.read_csv("instacart-market-basket-analysis/products.csv")

In [17]:
df_product.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [19]:
df_product.shape

(49688, 4)

In [20]:
df_dept = pd.read_csv("instacart-market-basket-analysis/departments.csv")

In [27]:
df_dept.shape

(21, 2)

In [28]:
df_dept.head(25)

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [24]:
query = """
            select
                  a.order_id,
                  a.product_id,
                  b.department_id
            from
                df_orders a
            join
                df_product b
            on 
                a.product_id = b.product_id
                
        """

In [25]:
df_orders_detailed = sqldf(query, globals())

In [29]:
df_orders_detailed.head()

,order_id,product_id,department_id
0,2,33120,16
1,2,28985,4
2,2,9327,13
3,2,45918,13
4,2,30035,13


In [30]:
df_orders_detailed.shape

(32434489, 3)

In [81]:
df_items = df_orders_detailed[df_orders_detailed['department_id'].isin([1,3,4,7,13,14,16,19,20])]

In [82]:
df_items.shape

(27520600, 3)

In [83]:
query = """
            select
                  product_id1,
                  department_id1,
                  product_id2,
                  department_id2,
                  count(*) as cnt
            from
            (
                select a.product_id as product_id1,
                       a.department_id as department_id1,
                       b.product_id as product_id2,
                       b.department_id as department_id2
                from 
                       df_items a
                join
                       df_items b
                on 
                       a.order_id = b.order_id
            ) a
            where
                product_id1 < product_id2
            and 
                department_id1 != department_id2
            group by
                product_id1,
                department_id1,
                product_id2,
                department_id2
            order by cnt desc
                
        """

In [84]:
df_grouped = sqldf(query, globals())

In [85]:
df_grouped.head()

,product_id1,department_id1,product_id2,department_id2,cnt
0,24852,4,27845,16,31641
1,13176,4,27845,16,26481
2,21137,4,27845,16,23813
3,24852,4,30489,20,18210
4,21903,4,27845,16,16944


In [86]:
df_grouped.shape

(19124682, 5)

In [96]:
query = """
            select
                  a.product_id1,
                  a.product_id2,
                  a.product_name1,
                  b.product_name as product_name2,
                  a.department_id1,
                  a.department_id2,
                  a.cnt
            from
            (
                select
                    a.product_id1,
                    a.product_id2,
                    b.product_name as product_name1,
                    a.department_id1,
                    a.department_id2,
                    a.cnt
                from
                    (select * from df_grouped) a
                join
                    (select * from df_product) b
                on 
                    a.product_id1 = b.product_id
                and
                    a.department_id1 = b.department_id
            ) a
            join
                (select * from df_product) b
            on
                a.product_id2 = b.product_id
            and
                a.department_id2 = b.department_id
        """

In [97]:
df_grouped_detailed = sqldf(query,globals())

In [110]:
df_grouped_detailed.head()

,product_id1,product_id2,product_name1,product_name2,department_id1,department_id2,cnt
0,24852,27845,Banana,Organic Whole Milk,4,16,31641
1,13176,27845,Bag of Organic Bananas,Organic Whole Milk,4,16,26481
2,21137,27845,Organic Strawberries,Organic Whole Milk,4,16,23813
3,24852,30489,Banana,Original Hummus,4,20,18210
4,21903,27845,Organic Baby Spinach,Organic Whole Milk,4,16,16944


In [109]:
df_grouped_detailed.tail()

,product_id1,product_id2,product_name1,product_name2,department_id1,department_id2,cnt
19124677,49644,49678,Feta Crumbles,Large Chicken & Cheese Taquitos,16,1,1
19124678,49644,49680,Feta Crumbles,All Natural Creamy Caesar Dressing,16,13,1
19124679,49654,49683,"Teriyaki Sauce, Sesame, Original",Cucumber Kirby,13,4,1
19124680,49671,49683,Milk Chocolate Drops,Cucumber Kirby,19,4,1
19124681,49683,49685,Cucumber Kirby,En Croute Roast Hazelnut Cranberry,4,1,1


In [102]:
df_grouped_detailed.shape

(19124682, 7)

In [103]:
df_grouped_detailed[(df_grouped_detailed['department_id1']!=4) & (df_grouped_detailed['department_id2']!=4)].shape

(14160692, 7)

In [108]:
df_grouped_detailed[(df_grouped_detailed['department_id1']!=4) & (df_grouped_detailed['department_id2']!=4)].head()

,product_id1,product_id2,product_name1,product_name2,department_id1,department_id2,cnt
140,27845,30489,Organic Whole Milk,Original Hummus,16,20,6020
286,27845,44632,Organic Whole Milk,Sparkling Water Grapefruit,16,7,4374
294,22035,30489,Organic Whole String Cheese,Original Hummus,16,20,4332
332,27086,44632,Half & Half,Sparkling Water Grapefruit,16,7,4105
351,30489,44632,Original Hummus,Sparkling Water Grapefruit,20,7,3999


In [134]:
query = """
            select 
                product_id,
                count(*) as cnt
            from
                df_orders
            group by
                product_id
        """

In [135]:
df_product_bought_count = sqldf(query,globals())

In [136]:
df_product_bought_count.head()

,product_id,cnt
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15


In [137]:
df_grouped_detailed_sample = df_grouped_detailed.head(50)

In [138]:
df_grouped_detailed_sample.head()

,product_id1,product_id2,product_name1,product_name2,department_id1,department_id2,cnt
0,24852,27845,Banana,Organic Whole Milk,4,16,31641
1,13176,27845,Bag of Organic Bananas,Organic Whole Milk,4,16,26481
2,21137,27845,Organic Strawberries,Organic Whole Milk,4,16,23813
3,24852,30489,Banana,Original Hummus,4,20,18210
4,21903,27845,Organic Baby Spinach,Organic Whole Milk,4,16,16944


In [139]:
df_product_bought_count_sample = df_product_bought_count.head(10)

In [140]:
df_product_bought_count_sample.head()

,product_id,cnt
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15


In [161]:
query = """
            select 
                product_id1,
                product_id2,
                product_name1,
                product_name2,
                department_id1,
                department_id2,
                cnt as item_intersection,
                cnt1+cnt2-cnt as item_union,
                (cnt*1.0)/(cnt1+cnt2-cnt) as ratio
                from
               (select 
                    a.*,
                    b.cnt as cnt2
                from
                    (select 
                        a.*,
                        b.cnt as cnt1
                    from
                        (select * from df_grouped_detailed_sample) a
                    join
                        (select * from df_product_bought_count_sample) b
                    on
                        a.product_id1 = b.product_id) a
                join
                    (select * from df_product_bought_count_sample) b
                on
                    a.product_id2 = b.product_id) a
        """
df_tmp = sqldf(query,globals())

In [162]:
df_tmp.head()

,product_id1,product_id2,product_name1,product_name2,department_id1,department_id2,item_intersection,item_union,ratio


In [163]:
query = """
            select 
                product_id1,
                product_id2,
                product_name1,
                product_name2,
                department_id1,
                department_id2,
                cnt as item_intersection,
                cnt1+cnt2-cnt as item_union,
                (cnt*1.0)/(cnt1+cnt2-cnt) as ratio
                from
                    (select 
                        a.*,
                        b.cnt as cnt2
                    from
                        (select 
                            a.*,
                            b.cnt as cnt1
                        from
                            (select * from df_grouped_detailed) a
                        join
                            (select * from df_product_bought_count) b
                        on
                            a.product_id1 = b.product_id) a
                    join
                        (select * from df_product_bought_count) b
                    on
                        a.product_id2 = b.product_id) a

        """

In [164]:
df_ratio = sqldf(query,globals())

In [165]:
df_ratio.head()

,product_id1,product_id2,product_name1,product_name2,department_id1,department_id2,item_intersection,item_union,ratio
0,24852,27845,Banana,Organic Whole Milk,4,16,31641,578829,0.054664
1,13176,27845,Bag of Organic Bananas,Organic Whole Milk,4,16,26481,490874,0.053947
2,21137,27845,Organic Strawberries,Organic Whole Milk,4,16,23813,378775,0.062868
3,24852,30489,Banana,Original Hummus,4,20,18210,525669,0.034642
4,21903,27845,Organic Baby Spinach,Organic Whole Milk,4,16,16944,362882,0.046693


In [166]:
df_ratio_sorted = df_ratio.sort_values('ratio',ascending=False)

In [174]:
!jupyter nbextension enable --py --sys-prefix qgrid
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension qgrid/extension...
      - Validating: OK
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [170]:
import qgrid

In [184]:
# qgrid.show_grid(df_ratio_sorted.head(50),
#                 grid_options={'fullWidthRows': True,
#                               'syncColumnCellResize': True,
#                               'forceFitColumns': True,
#                               'rowHeight': 40,
#                               'enableColumnReorder': True,
#                               'enableTextSelectionOnCells': True,
#                               'editable': True})

In [185]:
df_ratio_sorted[df_ratio_sorted.item_union > 500].head(50)

,product_id1,product_id2,product_name1,product_name2,department_id1,department_id2,item_intersection,item_union,ratio
2657,13269,44786,Oh My Yog! Pacific Coast Strawberry Trilayer Y...,Oh My Yog! Organic Wild Quebec Blueberry Cream...,16,19,1502,3627,0.414116
5497,6508,44786,Oh My Yog! Madagascar Vanilla Trilayer Yogyurt,Oh My Yog! Organic Wild Quebec Blueberry Cream...,16,19,984,3857,0.255121
7904,44786,48747,Oh My Yog! Organic Wild Quebec Blueberry Cream...,Oh My Yog! Gingered Pear Trilayer Yogurt,19,16,787,3238,0.243051
43714,13029,13287,Ready Pac Salad Santa Fe Caesar Bowls,Bistro Bowl Chicken Caesar Salad,20,4,254,1791,0.141820
20181,19005,36849,Jet-Puffed Marshmallows,Milk Chocolate Candy Bars,13,19,436,3296,0.132282
32274,36496,44786,Oh My Yog! Trilayer Yogurt Apple Cinnamon,Oh My Yog! Organic Wild Quebec Blueberry Cream...,16,19,316,2569,0.123005
139411,1080,33016,"Salad, Cobb",Chef Salad,4,20,105,933,0.112540
66042,21567,45977,Blueberry Almond Breakfast Bars,Breakfast Honey Oat Gluten Free Non GMO 5 Supe...,14,3,188,1687,0.111440
77673,13287,33016,Bistro Bowl Chicken Caesar Salad,Chef Salad,4,20,166,1616,0.102723
183973,11961,48390,Organic Nuts & Vanilla Beverage,Organic Nuts & Chocolate Beverage,7,19,84,846,0.099291


In [177]:
# df_ratio_sorted[df_ratio_sorted.item_union > 500].head(50).to_csv("r.csv",header=True,index=False)

In [ ]:
# df_ratio_sorted

In [186]:
# df_product[df_product['product_id']==13269]

In [187]:
# df_product[df_product['product_id']==44786]